### Develop LSTM Models For Univariate Time Series Forecasting

In [ ]:

# univariate lstm example
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import pandas as pd
import yfinance
from datetime import date,timedelta


In [ ]:
StartDate = (date.today()-timedelta(days=7)).strftime('%Y-%m-%d')
EndDate= (date.today().strftime('%Y-%m-%d'))
StartDate,EndDate

('2024-02-11', '2024-02-18')

In [ ]:
df=yfinance.download('AAPL', start=(date.today()-timedelta(days=7)).strftime('%Y-%m-%d'),
                     end=date.today()-timedelta(days=2),
                     interval='1m')
print(df.shape)
df.tail(15)

[*********************100%%**********************]  1 of 1 completed

(1559, 6)


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-02-15 15:45:00-05:00,182.270004,182.380005,182.240005,182.356003,182.356003,185989
2024-02-15 15:46:00-05:00,182.350006,182.389999,182.309998,182.380798,182.380798,113903
2024-02-15 15:47:00-05:00,182.384995,182.475006,182.360001,182.440002,182.440002,183468
2024-02-15 15:48:00-05:00,182.429993,182.460007,182.354996,182.425003,182.425003,116283
2024-02-15 15:49:00-05:00,182.425003,182.425003,182.324997,182.360001,182.360001,101240
2024-02-15 15:50:00-05:00,182.360001,182.589996,182.360001,182.580002,182.580002,323090
2024-02-15 15:51:00-05:00,182.580002,182.645004,182.449997,182.619995,182.619995,333013
2024-02-15 15:52:00-05:00,182.639999,182.649994,182.570007,182.574997,182.574997,171195
2024-02-15 15:53:00-05:00,182.580002,182.645004,182.509995,182.554993,182.554993,217760


In [ ]:
# preparing independent and dependent features
def prepare_data(timeseries_data, n_features):
	X, y =[],[]
	for i in range(len(timeseries_data)):
		# find the end of this pattern
		end_ix = i + n_features
		# check if we are beyond the sequence
		if end_ix > len(timeseries_data)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [ ]:
# define input sequence
timeseries_data = df['Open']
# choose a number of time steps
n_steps = 50
# split into samples
X, y = prepare_data(timeseries_data, n_steps)

In [ ]:
print(X),print(y)

[[188.41499329 188.43499756 188.27999878 ... 187.75       187.71000671
  187.71980286]
 [188.43499756 188.27999878 188.38000488 ... 187.71000671 187.71980286
  187.67500305]
 [188.27999878 188.38000488 188.48500061 ... 187.71980286 187.67500305
  187.69999695]
 ...
 [182.53999329 182.52999878 182.63999939 ... 182.57000732 183.5
  183.61500549]
 [182.52999878 182.63999939 182.49000549 ... 183.5        183.61500549
  183.66999817]
 [182.63999939 182.49000549 182.46000671 ... 183.61500549 183.66999817
  184.36999512]]
[187.67500305 187.69999695 187.69999695 ... 183.66999817 184.36999512
 184.11500549]


(None, None)

In [ ]:
X.shape

(1509, 50)

In [ ]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

### Building LSTM Model

In [ ]:

# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=50, verbose=1)


Epoch 1/50
48/48 [==============================] - 5s 53ms/step - loss: 3562680.0000
Epoch 2/50
48/48 [==============================] - 2s 49ms/step - loss: 6771157.5000
Epoch 3/50
48/48 [==============================] - 2s 48ms/step - loss: 3040748.7500
Epoch 4/50
48/48 [==============================] - 2s 44ms/step - loss: 1297728.1250
Epoch 5/50
48/48 [==============================] - 2s 51ms/step - loss: 131017.6172
Epoch 6/50
48/48 [==============================] - 3s 65ms/step - loss: 10831.9268
Epoch 7/50
48/48 [==============================] - 2s 45ms/step - loss: 1075.8998
Epoch 8/50
48/48 [==============================] - 2s 46ms/step - loss: 66.5478
Epoch 9/50
48/48 [==============================] - 3s 55ms/step - loss: 3.1299
Epoch 10/50
48/48 [==============================] - 4s 81ms/step - loss: 0.5933
Epoch 11/50
48/48 [==============================] - 3s 60ms/step - loss: 0.5325
Epoch 12/50
48/48 [==============================] - 2s 42ms/step - loss: 0.5315


In [ ]:
start='2024-02-16'
end=date.today()-timedelta(days=2)
start,end

('2024-02-16', datetime.date(2024, 2, 16))

In [ ]:
model.save('AAPLE_LSTM.h5')

### Predicting For the next 10 data

In [ ]:
df2=yfinance.download('AAPL', start='2024-02-15',
                     end='2024-02-16',
                     interval='1m')
print(df2.shape)
df2

[*********************100%%**********************]  1 of 1 completed

(390, 6)


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-02-15 09:30:00-05:00,183.214996,183.779999,183.149994,183.214996,183.214996,4168200
2024-02-15 09:31:00-05:00,183.169998,183.503601,183.059998,183.100006,183.100006,372357
2024-02-15 09:32:00-05:00,183.117996,183.274506,183.029999,183.119995,183.119995,213666
2024-02-15 09:33:00-05:00,183.125000,183.490005,183.110001,183.389999,183.389999,206582
2024-02-15 09:34:00-05:00,183.380096,183.463104,183.289993,183.300003,183.300003,175279
...,...,...,...,...,...,...
2024-02-15 15:55:00-05:00,183.500000,183.669998,182.960007,183.615005,183.615005,1179519
2024-02-15 15:56:00-05:00,183.615005,183.740005,183.369995,183.669998,183.669998,884530
2024-02-15 15:57:00-05:00,183.669998,184.490005,183.660004,184.360001,184.360001,1298572


In [ ]:
x_input = df2['Open']

In [ ]:
# demonstrate prediction for next 10 days
x_input = np.array(x_input)
temp_input = list(x_input)
lst_output = []
i = 0
while i < 20:
    if len(temp_input) >= n_steps:  # Ensure length is sufficient for reshaping
        x_input = np.array(temp_input[-n_steps:])  # Take the last n_steps elements
        print("{} day input {}".format(i, x_input))
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i, yhat))
        temp_input.append(yhat[0][0])
        lst_output.append(yhat[0][0])
        i = i + 1

    else:
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.append(yhat[0][0])
        lst_output.append(yhat[0][0])
        i=i+1


print(lst_output)

0 day input [182.49000549 182.46000671 182.52000427 182.58000183 182.52000427
 182.38999939 182.41499329 182.3500061  182.32090759 182.27000427
 182.22399902 182.25010681 182.25999451 182.22999573 182.25999451
 182.13000488 182.17680359 182.28999329 182.375      182.42999268
 182.38690186 182.47999573 182.50999451 182.43499756 182.46000671
 182.41999817 182.41999817 182.42999268 182.35220337 182.36999512
 182.30499268 182.36000061 182.23500061 182.22000122 182.32499695
 182.27000427 182.3500061  182.38499451 182.42999268 182.42500305
 182.36000061 182.58000183 182.63999939 182.58000183 182.57000732
 183.5        183.61500549 183.66999817 184.36999512 184.11500549]
0 day output [[182.39076]]
1 day input [182.46000671 182.52000427 182.58000183 182.52000427 182.38999939
 182.41499329 182.3500061  182.32090759 182.27000427 182.22399902
 182.25010681 182.25999451 182.22999573 182.25999451 182.13000488
 182.17680359 182.28999329 182.375      182.42999268 182.38690186
 182.47999573 182.509994

In [ ]:
timeseries_data

Datetime
2024-02-12 09:30:00-05:00    188.414993
2024-02-12 09:31:00-05:00    188.434998
2024-02-12 09:32:00-05:00    188.279999
2024-02-12 09:33:00-05:00    188.380005
2024-02-12 09:34:00-05:00    188.485001
                                ...    
2024-02-15 15:55:00-05:00    183.500000
2024-02-15 15:56:00-05:00    183.615005
2024-02-15 15:57:00-05:00    183.669998
2024-02-15 15:58:00-05:00    184.369995
2024-02-15 15:59:00-05:00    184.115005
Name: Open, Length: 1559, dtype: float64

In [ ]:
len(timeseries_data)

1559

In [ ]:
lst_output

[182.39076,
 182.37743,
 182.3684,
 182.27211,
 182.20047,
 182.19736,
 182.09117,
 182.12746,
 182.10445,
 182.13953,
 182.17198,
 182.10887,
 182.18542,
 182.07758,
 182.14833,
 182.2468,
 182.25258,
 182.40886,
 182.37947,
 182.13116]

In [ ]:
df2=yfinance.download('AAPL', start='2024-02-16',
                     end='2024-02-17',
                     interval='1m')
print(df2.shape)
df2

[*********************100%%**********************]  1 of 1 completed

(390, 6)


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-02-16 09:30:00-05:00,183.740005,183.740005,183.740005,183.740005,183.740005,3466801
2024-02-16 09:31:00-05:00,183.720001,183.839996,183.309998,183.479202,183.479202,240712
2024-02-16 09:32:00-05:00,183.479996,183.639999,183.320007,183.500000,183.500000,172233
2024-02-16 09:33:00-05:00,183.494995,183.759995,183.389999,183.710007,183.710007,160828
2024-02-16 09:34:00-05:00,183.695007,184.059998,183.695007,184.039993,184.039993,271552
...,...,...,...,...,...,...
2024-02-16 15:55:00-05:00,181.960007,182.089996,181.945007,182.054993,182.054993,273874
2024-02-16 15:56:00-05:00,182.054993,182.097504,182.014999,182.074997,182.074997,201150
2024-02-16 15:57:00-05:00,182.070007,182.184998,182.060104,182.184998,182.184998,236217


In [ ]:
dataframe = pd.DataFrame({"Actual":df2['Close'].head(20),"Prediction":lst_output})
dataframe

,Actual,Prediction
Datetime,,
2024-02-16 09:30:00-05:00,183.740005,182.390762
2024-02-16 09:31:00-05:00,183.479202,182.377426
2024-02-16 09:32:00-05:00,183.500000,182.368393
2024-02-16 09:33:00-05:00,183.710007,182.272110
2024-02-16 09:34:00-05:00,184.039993,182.200470
2024-02-16 09:35:00-05:00,184.131897,182.197357
2024-02-16 09:36:00-05:00,184.410294,182.091171
2024-02-16 09:37:00-05:00,184.781006,182.127457
2024-02-16 09:38:00-05:00,184.694000,182.104446
